In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors


import numpy as np


: 

In [34]:
from transformers import BertModel, BertTokenizer
import torch

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/Users/tejalnair/Desktop/BRH/impact-zero/myvenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
def get_embedding(text):
    # Encode text to get input IDs and attention mask
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # Get embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    # Use mean pooling to convert to a single vector
    embeddings = outputs.last_hidden_state.mean(1)
    return embeddings

In [24]:
df = pd.read_csv('/Users/tejalnair/Desktop/BRH/impact-zero/NetZero - Sheet3.csv')

In [26]:
df.columns = ['Action', 'Impact_Score']

In [36]:
df['embeddings'] = df['Action'].apply(lambda x: get_embedding(x).numpy()[0])

In [37]:
embedding_matrix = np.vstack(df['embeddings'])

In [39]:
nn = NearestNeighbors(n_neighbors=1, metric='cosine')
nn.fit(embedding_matrix)

NearestNeighbors(metric='cosine', n_neighbors=1)

In [40]:
def predict_score(action):
    embedding = get_embedding(action)
    distance, indices = nn.kneighbors(embedding)
    closest_action = df.iloc[indices[0][0]]['Action']
    predicted_score = df.iloc[indices[0][0]]['Impact_Score']
    return closest_action, predicted_score

In [44]:
new_action = "chatgpt"
closest_action, impact_score = predict_score(new_action)
print(f"Closest action: {closest_action}, Impact score: {impact_score}")

Closest action: tvs, Impact score: 5


In [27]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Action'])


In [28]:
nn_model = NearestNeighbors(n_neighbors=1, metric='cosine')
nn_model.fit(X)

# Function to find nearest action and retrieve corresponding impact score
def get_impact_score(new_action):
    new_action_vec = vectorizer.transform([new_action])
    distance, indices = nn_model.kneighbors(new_action_vec)
    closest_action = df.iloc[indices[0][0]]['Action']
    closest_score = df.iloc[indices[0][0]]['Impact_Score']
    return closest_action, closest_score

In [32]:

# Example usage
new_action = "phone"
closest_action, impact_score = get_impact_score(new_action)
print(f"Closest action: {closest_action}, Impact score: {impact_score}")

Closest action: Car (electric), Impact score: 4


In [4]:
df.rename(columns={'Unnamed: 5': 'material_type', 'Unnamed: 6': 'total_eco_costs'}, inplace=True)

In [5]:
df.dropna(subset=['material_type', 'total_eco_costs'], inplace=True)

In [6]:
df['total_eco_costs'] = pd.to_numeric(df['total_eco_costs'], errors='coerce')
df.dropna(subset=['total_eco_costs'], inplace=True)

In [7]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,material_type,total_eco_costs,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 26,Sources of LCI:,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35
1,"A.010.02.101.240318 Leather Chrome tanning, hi...",agriculture,A.010.02.101,"Materials, agricultural, animal production",kg,"Leather Chrome tanning, hides from Argentia",5.8400,5.84,1.38,1.13,...,NaN,LCA for leather (chromium process). Hides assu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"A.010.02.102.240318 Leather Vegetable tanning,...",agriculture,A.010.02.102,"Materials, agricultural, animal production",kg,"Leather Vegetable tanning, hides from Europe",2.7400,2.74,0.17,0.13,...,NaN,LCA for leather (chromium process). Hides assu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"A.010.02.103.230701 Wool (Australia, transport...",agriculture,A.010.02.103,"Materials, agricultural, animal production",kg,"Wool (Australia, transported to Rotterdam)",10.8800,10.88,0.03,0.06,...,NaN,Wiedemann et al.. Application of life cycle as...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A.010.02.104.230701 Wool at farm Australia,agriculture,A.010.02.104,"Materials, agricultural, animal production",kg,Wool at farm Australia,10.8000,10.80,0.02,0.05,...,NaN,Wiedemann et al.. Application of life cycle as...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,A.010.04.101.230701 Crude palm kernel oil refined,agriculture,A.010.04.101,"Materials, agricultural, others",kg,Crude palm kernel oil refined,0.7800,0.78,0.01,0.63,...,NaN,recalculated from USLCI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479,Q.010.40.022.240709 Trichloroethylene (C2HCl3)...,scope 1 & downsrtream,Q.010.40.022,Scope 1 and downstream direct emissions,kg,"Trichloroethylene (C2HCl3), direct emissions (...",3.3912,3.3912,3.3852,0.0002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2480,"Q.010.40.023.240709 Turpentine, direct emissio...",scope 1 & downsrtream,Q.010.40.023,Scope 1 and downstream direct emissions,kg,"Turpentine, direct emissions (scope1 and downs...",0.0021,0.0021,0.0000,0.0021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2481,"Q.010.40.023.240709 White spirit, direct emiss...",scope 1 & downsrtream,Q.010.40.023,Scope 1 and downstream direct emissions,kg,"White spirit, direct emissions (scope1 and dow...",0.0487,0.0487,0.0486,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2482,"Q.010.40.024.240709 Xylene (C6H4(CH3)2), direc...",scope 1 & downsrtream,Q.010.40.024,Scope 1 and downstream direct emissions,kg,"Xylene (C6H4(CH3)2), direct emissions (scope1 ...",0.0282,0.0282,0.0280,0.0002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print("Number of unique categories in 'material_type':", df['material_type'].nunique())

Number of unique categories in 'material_type': 2269


In [13]:
tfidf = TfidfVectorizer(max_features=1000)  # You can adjust 'max_features' as needed

# Fit and transform the material_type column
X = tfidf.fit_transform(df['material_type'].astype(str))

# Prepare the target variable
y = df['total_eco_costs'].astype(float)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
# Initialize the Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [16]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model using Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 1922.8057520914374


In [10]:
# Prepare the feature using One-Hot Encoding
encoder = OneHotEncoder()  # sparse=False means the output will be a dense numpy array
material_encoded = encoder.fit_transform(df[['material_type']]).astype(str)  # Fit and transform the data


# Convert the array returned by OneHotEncoder back to a DataFrame
encoded_df = pd.DataFrame(material_encoded, columns=encoder.get_feature_names_out(['material_type']), index=df.index)

# Add the encoded DataFrame back to the original DataFrame
df = df.join(encoded_df)


ValueError: Shape of passed values is (2294, 1), indices imply (2294, 2291)